Copyright 2021-2022 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 06: Multi-task with multiple readouts (example 2)

In [1]:
import sys
import time
import numpy as np
import mindspore as ms
from mindspore import nn
from mindspore import Tensor
from mindspore import context
from mindspore import dataset as ds
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

from cybertron import Cybertron
from cybertron import MolCT
from cybertron.train import MAE, MLoss
from cybertron.train import WithLabelLossCell, WithLabelEvalCell
from cybertron.train import TrainMonitor
from cybertron.train import TransformerLR

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

[WARNING] ME(15775:140549731665728,MainProcess):2022-08-10-17:33:36.300.071 [mindspore/run_check/_check_version.py:137] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
data_name = sys.path[0] + '/dataset_qm9_normed_'
train_file = data_name + 'trainset_1024.npz'
valid_file = data_name + 'validset_128.npz'

train_data = np.load(train_file)
valid_data = np.load(valid_file)

# diplole,polarizability,HOMO,LUMO,gap,R2,zpve,capacity
idx = [0, 1, 2, 3, 4, 5, 6, 11]

num_atom = int(train_data['num_atoms'])
scale = Tensor(train_data['scale'][idx], ms.float32)
shift = Tensor(train_data['shift'][idx], ms.float32)
ref = Tensor(train_data['type_ref'][:, idx], ms.float32)

In [3]:
mod = MolCT(
    cutoff=1,
    n_interaction=3,
    dim_feature=128,
    n_heads=8,
    activation='swish',
    max_cycles=1,
    length_unit='nm',
)

In [4]:
net = Cybertron(mod, readout='graph', dim_output=[1, 1, 3, 1, 1, 1],
                num_atoms=num_atom, length_unit='nm')
net.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
--------------------------------------------------------------------------------
    Deep molecular model:  MolCT
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): LogGaussianBasis
          Minimum distance: 0.04 nm
          Maximum distance: 1.0 nm
          Reference distance: 1.0 nm
          Log Gaussian begin: -3.218876
          Log Gaussian end: 0.006724119
          Interval for log Gaussian: 0.0512
          Sigma for log gaussian: 0.3
          Number of basis functions: 64
          Rescale the range of RBF to (-1,1).
       Calculate distance: Yes
       Calculate bond: No
       Feature dimension: 128
-----------------------------------------------------------------------

In [5]:
tot_params = 0
for i, param in enumerate(net.get_parameters()):
    tot_params += param.size
    print(i, param.name, param.shape)
print('Total parameters: ', tot_params)

0 model.atom_embedding.embedding_table (64, 128)
1 model.dis_filter.linear.weight (128, 64)
2 model.dis_filter.linear.bias (128,)
3 model.dis_filter.residual.nonlinear.mlp.0.weight (128, 128)
4 model.dis_filter.residual.nonlinear.mlp.0.bias (128,)
5 model.dis_filter.residual.nonlinear.mlp.1.weight (128, 128)
6 model.dis_filter.residual.nonlinear.mlp.1.bias (128,)
7 model.interactions.0.positional_embedding.norm.gamma (128,)
8 model.interactions.0.positional_embedding.norm.beta (128,)
9 model.interactions.0.positional_embedding.x2q.weight (128, 128)
10 model.interactions.0.positional_embedding.x2k.weight (128, 128)
11 model.interactions.0.positional_embedding.x2v.weight (128, 128)
12 model.interactions.0.multi_head_attention.output.weight (128, 128)
13 model.interactions.1.positional_embedding.norm.gamma (128,)
14 model.interactions.1.positional_embedding.norm.beta (128,)
15 model.interactions.1.positional_embedding.x2q.weight (128, 128)
16 model.interactions.1.positional_embedding.x2k.

In [6]:
n_epoch = 8
repeat_time = 1
batch_size = 32

In [7]:
ds_train = ds.NumpySlicesDataset(
    {'R': train_data['R'], 'Z': train_data['Z'], 'E': train_data['E'][:, idx]}, shuffle=True)
ds_train = ds_train.batch(batch_size, drop_remainder=True)
ds_train = ds_train.repeat(repeat_time)

In [8]:
ds_valid = ds.NumpySlicesDataset(
    {'R': valid_data['R'], 'Z': valid_data['Z'], 'E': valid_data['E'][:, idx]}, shuffle=False)
ds_valid = ds_valid.batch(128)
ds_valid = ds_valid.repeat(1)

In [9]:
loss_network = WithLabelLossCell('RZE', net, nn.MAELoss())
eval_network = WithLabelEvalCell('RZE', net, nn.MAELoss(), scale=scale, shift=shift, type_ref=ref)

WithLabelLossCell with input type: RZE
WithLabelEvalCell with input type: RZE
   with scaleshift for training and evaluate dataset:
   Output.            Scale           Shift        Mode
   0:        1.503183e+00    2.672827e+00       graph
   1:        8.173762e+00    7.528103e+01       graph
   2:        5.767056e+01   -6.306687e+02       graph
   3:        1.229870e+02    3.108809e+01       graph
   4:        1.238940e+02    6.617564e+02       graph
   5:        2.804632e+02    1.189402e+03       graph
   6:        8.699451e+01    3.914438e+02       graph
   7:        6.082039e+00   -2.213512e+01       graph
   with reference value for atom types:
   Type     Label0    Label1    Label2    Label3    Label4    Label5    Label6    Label7
   0:        0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   1:        0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  2.98e+00
   2:        0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00

In [10]:
lr = TransformerLR(learning_rate=1., warmup_steps=4000, dimension=128)
optim = nn.Adam(params=net.trainable_params(), learning_rate=lr)

In [11]:
eval_mae = 'EvalMAE'
atom_mae = 'AtomMAE'
eval_loss = 'Evalloss'
model = Model(loss_network, optimizer=optim, eval_network=eval_network,
              metrics={eval_mae: MAE([1, 2], reduce_all_dims=False),
                       atom_mae: MAE([1, 2, 3], reduce_all_dims=False, averaged_by_atoms=True),
                       eval_loss: MLoss(0)},)


In [12]:
outdir = 'Tutorial_C06'
outname = outdir + '_' + net.model_name
record_cb = TrainMonitor(model, outname, per_step=32, avg_steps=32,
                         directory=outdir, eval_dataset=ds_valid, best_ckpt_metrics=eval_loss)

In [13]:
config_ck = CheckpointConfig(save_checkpoint_steps=32, keep_checkpoint_max=64, append_info=[net.hyper_param])
ckpoint_cb = ModelCheckpoint(prefix=outname, directory=outdir, config=config_ck)

In [14]:
np.set_printoptions(linewidth=200)

print("Start training ...")
beg_time = time.time()
model.train(n_epoch, ds_train, callbacks=[
    record_cb, ckpoint_cb], dataset_sink_mode=False)
end_time = time.time()
used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print("Training Fininshed!")
print("Training Time: %02d:%02d:%02d" % (h, m, s))

[WARNING] ME(15775:140549731665728,MainProcess):2022-08-10-17:33:54.865.42 [mindspore/train/model.py:1097] For TrainMonitor callback, {'begin', 'epoch_end', 'step_end'} methods may not be supported in later version, Use methods prefixed with 'on_train' or 'on_eval' instead when using customized callbacks.


Start training ...
Epoch: 1, Step: 32, Learning_rate: 1.0830951e-05, Last_Loss: 0.8817136, Avg_loss: 0.9816328808665276, EvalMAE: [  1.354305    7.0573864  73.874756  101.67707    95.05714   201.4472     63.426956    4.721528 ], AtomMAE: [ 0.07720656  0.42350703  4.1470666   6.198917    5.4334555  10.866326    3.9789188   0.2760198 ], Evalloss: 0.8578746914863586
Epoch: 2, Step: 64, Learning_rate: 2.2011289e-05, Last_Loss: 0.7452917, Avg_loss: 0.8000287134200335, EvalMAE: [  1.2932307   6.6266966  45.27566    87.648575   94.00476   191.88496    53.885666    4.3978643], AtomMAE: [ 0.07598674  0.39586204  2.769402    5.1205387   5.3762245  10.541361    3.18947     0.26172012], Evalloss: 0.744278073310852
Epoch: 3, Step: 96, Learning_rate: 3.3191627e-05, Last_Loss: 0.6801481, Avg_loss: 0.7273177430033684, EvalMAE: [  1.2349188   5.7119527  42.25477    82.20943    91.34517   187.2038     44.88445     3.776333 ], AtomMAE: [ 0.07238529  0.33933985  2.5631635   4.847484    5.2437115  10.15517